## 导入包

In [1]:
import jiagu
from wechatsogou import WechatSogouAPI
import wechatsogou

## 1.获取短链接

In [2]:
import requests
import json
def get_short_url(long_url):
    host = 'https://dwz.cn'
    path = '/admin/v2/create'
    url = host + path
    method = 'POST'
    content_type = 'application/json'

    # TODO: 设置Token
    token='9e7d0fa08024c067ac6c06559c6825d6'

    # TODO：设置待创建的长网址
    bodys = {'url':long_url}
    # 配置headers
    headers = {'Content-Type':content_type, 'Token':token}
    # 发起请求
    response = requests.post(url=url, data=json.dumps(bodys), headers=headers)
# 读取响应
#  print(response.text)
    return response.json()['ShortUrl']

## 2.extract 正文文本

In [3]:
# -*- encoding: utf-8 -*-
import re
import requests
from collections import Counter
from bs4 import BeautifulSoup


'''
    获取html
'''
def get_html(url):
    obj = requests.get(url)
    return obj.text

'''
    过滤各类标签
    html_str:html字符串
    flag:是否移除所有标签
    return:过滤标签后的html字符串
'''
def filter_tags(html_str, flag):
    html_str = re.sub('(?is)<!DOCTYPE.*?>', '', html_str)
    html_str = re.sub('(?is)<!--.*?-->', '', html_str) #remove html comment
    html_str = re.sub('(?is)<script.*?>.*?</script>', '', html_str) #remove javascript
    html_str = re.sub('(?is)<style.*?>.*?</style>', '', html_str) #remove css
    html_str = re.sub('(?is)<a[\t|\n|\r|\f].*?>.*?</a>', '', html_str)  # remove a
    html_str = re.sub('(?is)<li[^nk].*?>.*?</li>', '', html_str)  # remove li
    #html_str = re.sub('&.{2,5};|&#.{2,5};', '', html_str) #remove special char
    if flag:
        html_str = re.sub('(?is)<.*?>', '', html_str) #remove tag
    return html_str

'''
    根据文本块密度获取正文
    html_str:网页源代码
    return：正文文本
'''
def extract_text_by_block(html_str):
    html = filter_tags(html_str, True)
    lines = html.split('\n')
    blockwidth = 3
    threshold = 86
    indexDistribution = []
    for i in range(0, len(lines) - blockwidth):
        wordnum = 0
        for j in range(i, i + blockwidth):
            line = re.sub("\\s+", '', lines[j])
            wordnum += len(line)
        indexDistribution.append(wordnum)
    startindex = -1
    endindex = -1
    boolstart = False
    boolend = False
    arcticle_content = []
    for i in range(0, len(indexDistribution) - blockwidth):
        if (indexDistribution[i] > threshold and boolstart is False):
            if indexDistribution[i + 1] != 0 or indexDistribution[i + 2] != 0 or indexDistribution[i + 3] != 0:
                boolstart = True
                startindex = i
                continue
        if boolstart is True:
            if indexDistribution[i] == 0 or indexDistribution[i + 1] == 0:
                endindex = i
                boolend = True
        tmp = []
        if boolend is True:
            for index in range(startindex, endindex + 1):
                line = lines[index]
                if len(line.strip()) < 5:
                    continue
                tmp.append(line.strip() + '\n')
            tmp_str = ''.join(tmp)
            if u"Copyright" in tmp_str or u"版权所有" in tmp_str:
                continue
            arcticle_content.append(tmp_str)
            boolstart = False
            boolend = False
    return ''.join(arcticle_content)

'''
    全网页查找根据文本块密度获取的正文，获取文本父级标签内的正文，目的是提高正文准确率
    html:网页html
    article:根据文本块密度获取的正文
    return:正文文本
'''
def extract_text_by_tag(html_str, article):
    lines = filter_tags(html_str, False)
    soup = BeautifulSoup(lines, 'lxml')
    p_list = soup.find_all('p')
    p_in_article = []
    for p in p_list:
        if p.text.strip() in article:
            p_in_article.append(p.parent)
    tuple = Counter(p_in_article).most_common(1)[0]
    article_soup = BeautifulSoup(str(tuple[0]), 'xml')
    return remove_space(article_soup.text)

'''
    移除字符串中的空白字符
'''
def remove_space(text):
    text = re.sub("[\t\r\n\f]", '', text)
    return text

'''
    抽取正文
    html_str:网页源代码
    return：正文文本
'''
def extract(url):
    html_str = get_html(url)
    article_temp = extract_text_by_block(html_str)
    try:
        article = extract_text_by_tag(html_str, article_temp)
    except:
        article = article_temp
    return article


## 3.转语音模块

In [4]:
from aip import AipSpeech
import time
 
""" 你的 APPID AK SK """
APP_ID = '10851881'
API_KEY = 'mlZtwyQ8Gtcsksr7KDYY85f2'
SECRET_KEY = 'qOpcErgd1uN0I65iZYOF8zCqTIDQE9hD'

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

def convert_to_mp3(content):
    '''
    content_type:str
    
    '''
    title=time.strftime("%Y-%m-%d", time.localtime()) 
    result  = client.synthesis(content, 'zh', 1, {
        'vol': 5,'per':3,'spd':6
    })

    # 识别正确返回语音二进制 错误则返回dict 参照下面错误码
    if not isinstance(result, dict):
        with open('./aicar_mp3/{}.mp3'.format(title), 'wb') as f:
            f.write(result)

## 4.常量和主函数

In [12]:
# 壁挂炉煤改气
# gzh_list=['喜德瑞BDR','squirrel小松鼠壁挂炉','天燃气','供暖信息报','暖通空调',' 天然气行业观察','煤改气','供热供暖','现代家电网','煤改电煤改气']

In [6]:
gzh_list = ['焉知自动驾驶', '大数据文摘','量子位','AI汽车制造业','Linux中国','高工智能汽车', '智车科技','42号车库','建约车评','机器之心']
ws_api = wechatsogou.WechatSogouAPI()
mp3_text=[]

def main():
    count=0
    for gzh in gzh_list:
        articles = ws_api.get_gzh_article_by_history(gzh)
    #     print(len(articles))
        url=articles['article'][0]['content_url']
        text= extract(url) #  基于行块分布的html正文抽取
        key_info= ''.join(jiagu.summarize(text,1))  #用了jiagu的文本摘要方法
        mp3_text.append(key_info)
        title=articles['article'][0]['title']
        count+=1
#         content=str(count)+'、'+title+'\n'+'链接：'+str(get_short_url(url))+'\n'+'摘要：'+key_info
#         with open('./aicar_text/{}.txt'.format('test'), 'w+') as f:
#             f.write(content,ensure_ascii=False)
        
        print(count,'、','**'+title+'**','\n\n','链接：',get_short_url(url),'\n\n',r'<font color=blue>摘要</font>：',key_info,'\n\n')
    text=''.join(mp3_text).replace('\xa0','')  #  去除干扰字符
    convert_to_mp3(text)  
main()

1 、 **手把手教你分析自动驾驶功能安全（一）** 

 链接： https://dwz.cn/R3cxDel9 

 <font color=blue>摘要</font>： 1、功能安全管理功能安全管理（FSM）是质量管理（QM）的延伸，主要包含如下功能：Ø 定义安全生命周期模型Ø 建立和促进公司安全文化Ø 定义所有相关部门、人员的安全责任Ø 保证必要的人员能力、资质Ø 保证足够的质量管理活动针对产品开发而言，功能安全管理的目标是：Ø 形成公司级功能安全规范和流程Ø 制定出功能安全培训指南Ø 功能安全机制集成到公司已在执行的质量管理体系当中2、 概念阶段功能安全概念阶段是整个自动驾驶系统开发功能安全分析最重要的节点，包括了四个阶段进行相应的工作：2.1 概念阶段2.1.1、对象相关项功能安全分析是基于对系统对象定义的，对象定义是对自动驾驶系统工作必要的相关项定义和描述，以确保安全生命周期中定义的每项活动都能得到准确实施，在相关项的定义过程中需要考虑行业专家意见（如设计中的建议与边界划分），同时必须考虑如下相关需求：Ø 整车级功能性的需求 Ø 环境要求（EMC，温度，水，尘… 


2 、 **MIT开发数据分析师的“无界画布”：实时交互预测，可拖拽的AutoML系统** 

 链接： https://dwz.cn/jLInE0zV 

 <font color=blue>摘要</font>： 而这，得益于一项新的AutoML组件：VDS迄今为止最快的交互式AutoML工具——VDSVDS首先出现在ACM SIGMOD会议上发表的一篇论文中，利用的是现在比较火的人工智能技术——自动机器学习(AutoML)，它可以让没有数据科学知识的人根据他们的数据集训练AI模型进行预测。 


3 、 **深挖之后吓一跳，谷歌AI专利何止一个dropout，至少30项今日生效** 

 链接： https://dwz.cn/WwJSCBoi 

 <font color=blue>摘要</font>： 而且， 并不只是谷歌本身展开了行动，它旗下的DeepMind，此前也申请了一大批AI专利，看名字都非常吓人，强化学习系统、循环神经网络尽在其中，列表如下：其中，”用神经网络生成音频“专利已经在美国生效，2037年9月6日到期。 


4 、 **【好文推荐】中国造车能力进化史** 

### 第四步完了将print的结果 直接复制到 https://lab.lyric.im/wxformat/ ,再复制到公众号即可

##  5.封面图裁剪opencv PIL

In [19]:
# -*- coding: utf-8 -*-
import os
import cv2
import numpy as np
import time
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime, timedelta
import shutil

pic_source_dir = os.path.join(os.getcwd(), "news_pic_source\\")  # 原图路径
pic_crop_dir = os.path.join(os.getcwd(), "news_pic_crop\\")  # 裁剪后的图片路径
pic_font_dir = os.path.join(os.getcwd(), "news_pic_font\\")  # 加字后的图片路径
start_date = "20190627"  # 绘制图片的其起始日期


# 判断文件夹是否存在，不存在则新建
def is_dir_existed(path, mkdir=True):
    if mkdir:
        if not os.path.exists(path):
            os.makedirs(path)
    else:
        return os.path.exists(path)


# opencv读取中文路径名会乱码
def cv_imread(file_path):
    cv_img = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), -1)
    return cv_img


# opencv写入中文路径名会乱码
def cv_imwrite(img, file_path):
    cv2.imencode('.jpg', img)[1].tofile(file_path)


# 把原图裁剪为多个小图(900*383)
def crop_little_pic(pic_path):
    img = cv_imread(pic_path)
    (sh, sw) = img.shape[:2]
    # 将图片的宽设置为900，高则按比例缩放
    res = cv2.resize(img, (900, round(sh * (900 / sw))))
    # 获取缩放后的高和宽，判断图片可裁剪的张数
    (ch, cw) = res.shape[:2]
    crop_pic_count = int(ch / 383)
    # 计算Y轴偏移
    start_y = int(ch % 383 / 2)
    # 根据图片的张数来决定怎么裁剪
    for i in range(0, crop_pic_count):
        crop_img = res[383 * i + start_y: 383 * (i + 1) + start_y, 0:900]
        cv_imwrite(crop_img, os.path.join(pic_crop_dir, str(int(round(time.time() * 1000))) + '.jpg'))


# 绘制文字
def draw_text(pic_path, date):
    img = cv_imread(pic_path)
    # 将图片从OpenCv格式转为PIL格式
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # 加载字体（字体文件名，字体大小）
    title_font = ImageFont.truetype('simhei.ttf', 52)
    date_font = ImageFont.truetype('simhei.ttf', 44)
    # 绘制文字的位置
    title_pos = (236, 110)
    date_pos = (316, 192)
    # 绘制内容
    title_content = u"『AI车圈速读』"
    date_content = u"第%s期" % date[2:]
    # 绘制
    draw = ImageDraw.Draw(img_pil)
    draw.text(title_pos, title_content, font=title_font, fill=(255, 255, 255))
    draw.text(date_pos, date_content, font=date_font, fill=(255, 255, 255))
    img_open_cv = cv2.cvtColor(np.asarray(img_pil), cv2.COLOR_RGB2BGR)
    cv_imwrite(img_open_cv, os.path.join(pic_font_dir, date[2:] + '.jpg'))


# 遍历获得某类文件路径列表
def fetch_file_path(path, file_type):
    file_list = []
    f = os.listdir(path)
    for i in f:
        if i.endswith(file_type):
            file_list.append(os.path.join(path, i))
    return file_list


# 构造生成日期列表
def init_date_list(begin_date, count):
    d_list = []
    begin_date = datetime.strptime(begin_date, "%Y%m%d")
    end_date = datetime.strptime((datetime.now() + timedelta(days=count)).strftime("%Y%m%d"), "%Y%m%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y%m%d")
        d_list.append(date_str)
        begin_date += timedelta(days=1)
    return d_list


if __name__ == '__main__':
    is_dir_existed(pic_source_dir)
    while True:
        choice = input(
            "%s\n请输入你想进行的操作\n1.进行图片裁剪\n2.图片加字\n3.清空裁剪文件夹\n4.清空加字文件夹\n5.退出程序\n%s\n" % ('=' * 32, '=' * 32))
        if choice == '1':
            is_dir_existed(pic_crop_dir)
            pic_path_list = fetch_file_path(pic_source_dir, ".jpg")
            if len(pic_path_list) == 0:
                print("原图文件夹中无图片，请先添加图片！")
            else:
                print("开始批量裁剪...")
                begin = datetime.now()
                for pic in pic_path_list:
                    crop_little_pic(pic)
                end = datetime.now()
                print("批量裁剪完毕，生成图片：%d张，耗时：%s秒" % (len(fetch_file_path(pic_crop_dir, ".jpg")), (end - begin).seconds))
        elif choice == '2':
            is_dir_existed(pic_font_dir)
            crop_path_list = fetch_file_path(pic_crop_dir, ".jpg")
            date_list = init_date_list(start_date, len(crop_path_list) + 1)
            if len(crop_path_list) == 0:
                print("裁剪文件夹中无图片，请先生成裁剪图片！")
            else:
                print("开始批量加字...")
                begin = datetime.now()
                print(len(crop_path_list), len(date_list))
                for i in range(len(crop_path_list)):
                    draw_text(crop_path_list[i], date_list[i])
                end = datetime.now()
                print("批量加字完毕，处理图片：%d张，耗时：%s秒" % (len(fetch_file_path(pic_font_dir, ".jpg")), (end - begin).seconds))
        elif choice == '3':
            if is_dir_existed(pic_crop_dir, False):
                shutil.rmtree(pic_crop_dir)
                print("文件夹删除成功！")
            else:
                print("文件夹不存在，删除失败~")
        elif choice == '4':
            if is_dir_existed(pic_font_dir, False):
                shutil.rmtree(pic_font_dir)
                print("文件夹删除成功！")
            else:
                print("文件夹不存在，删除失败~")
        elif choice == '5':
            exit("退出程序~")
        else:
            print("错误序号，请确认后重新输入!!!")


请输入你想进行的操作
1.进行图片裁剪
2.图片加字
3.清空裁剪文件夹
4.清空加字文件夹
5.退出程序
2
开始批量加字...
33 35
批量加字完毕，处理图片：33张，耗时：0秒
请输入你想进行的操作
1.进行图片裁剪
2.图片加字
3.清空裁剪文件夹
4.清空加字文件夹
5.退出程序
5
请输入你想进行的操作
1.进行图片裁剪
2.图片加字
3.清空裁剪文件夹
4.清空加字文件夹
5.退出程序
5
请输入你想进行的操作
1.进行图片裁剪
2.图片加字
3.清空裁剪文件夹
4.清空加字文件夹
5.退出程序
5


KeyboardInterrupt: 